In [82]:
import javalang
from javalang.tree import *
import itertools

# read
def read_program(filename):
    f =  open(filename, 'r')
    program = f.read()
    f.close();
    return program

from enum import Enum

class Param_GC(Enum):
    pass_forward = -1
    does_not_change = 1
    does_change = 0
    

class Variable_GC(Enum):
    pass_forward = -1
    garbage = 1
    not_garbage = 0

In [379]:
class Scope:
    def __init__(self, parent = None):
        self.local_variables = dict()
        self.tracked_variables = dict()
        self.parent = parent
    
    def add_local_variable(self, var):
        self.local_variables[var.name] = var
    
    def get_local_variable(self, var_name):
        return self.local_variables[var_name]
    
    def get_parent(self):
        return self.parent

    def make_trackable(self, var_name):
        var = self.get_local_variable(var_name)
        if var is not None:
            self.tracked_variables[var_name] = var
        else:
            raise(Exception("Can't make variable", var_name, "trackable because it's not local"))
    
    def is_tracked(self, var):
        tracked_vars = self.tracked_variables
        scope = self
        while tracked_vars.get(var, None) is None:
            if scope.get_parent():
                scope = scope.get_parent()
            else:
                return False
        return True
    
    def get_trackable(self):
        return set(map(lambda item: item[0], self.tracked_variables.items()))

In [309]:
class MethodInfo:
    def __init__(self, title, params):
        self.title = title
        self.input_param_names = [param.name for param in params]
        self.input_param_table = dict()
        self.scope = None

        for name in self.input_param_names:
            self.input_param_table[name] = Param_GC.not_sure

    def change_gc(self, param_name, new_gc):
        if new_gc_state not in Param_GC:
            raise Exception("Couldn't assign", new_gc, "type to param", param_name)
        param = self.input_param_table.get(param_name, None)
        if param is not None:
            self.input_param_table[param_name] = new_gc
        else:
            print("There is no param", param_name, "in method", self.title)
        
    def get_gc(self, param_name):
        return self.input_param_table[param_name]
    
    def attach_scope(self, scope):
        self.scope = scope
        
    def does_change_gc(self, var):
        return self.input_param_table[var]

class ClassInfo:
    def __init__(self, title, fields, parent = None):
        self.title = title
        self.methods = dict()
        self.parent = parent
        self.fields = [decl.name for decl in itertools.chain(*[field.declarators for field in fields])]
        
    def get_method(self, title):
        return self.methods[title]
    
    def add_method(self, method):
        self.methods[method.title] = method

In [408]:
class VariableAlias:
    def __init__(self, name, var_type=None):
        self.name = name
        self.parents = set([])
        self.gc = Variable_GC.garbage
        self.type = var_type
        
    def add_parent(self, parent):
        self.parents.add(parent)
        
    def change_gc(self, new_gc):
        if new_gc not in Variable_GC:
            raise Exception("Couldn't assign", new_gc, "type to param", self.name)
        else:
            self.gc = new_gc
            for p in self.parents:
                p.change_gc(new_gc)

In [154]:
def get_class_table(tree):
    class_nodes = map(lambda item: item[1], tree.filter(ClassDeclaration))
    class_table = dict()
    for class_node in class_nodes:
        cl = ClassInfo(class_node.name, class_node.fields)
        for method in class_node.methods:
            mn = MethodInfo(method.name, method.parameters)
            mn.attach_scope(process_method(method, Scope()))
            cl.add_method(mn)
        class_table[cl.title] = cl
    return class_table

In [354]:
def process_method(method, class_scope):
    scope = Scope(class_scope)
    
    for arg in method.parameters:
        scope.add_local_variable(VariableAlias(arg.name, arg.type))
        scope.make_trackable(arg.name)
    
    for cm in method.body:
        #print(cm)
        res = process_expression(cm, scope)
    return scope

In [476]:
def process_expression(exp, scope):
    #print(exp)
    if exp is BinaryOperation:
        pass
    elif isinstance(exp, LocalVariableDeclaration):
        return process_local_variable_declaration(exp, scope)
    elif isinstance(exp, VariableDeclarator):
        return process_variable_declarator(ext, scope)
    elif exp is IfStatement:
        pass
    elif exp is ForStatement:
        pass
    elif exp is MethodInvocation:
        pass
    elif isinstance(exp, Literal):
        return process_literal(exp, scope)
    elif isinstance(exp, MemberReference):
        return process_member_reference(exp, scope)
    elif isinstance(exp, ClassCreator):
        return process_class_creator(exp, scope)
    elif isinstance(exp, ReturnStatement):
        return process_return_statemnet(exp, scope)
    elif isinstance(exp, StatementExpression):
        process_statement_expression(exp, scope)
    elif isinstance(exp, MethodInvocation):
        process_method_invocation(exp, scope)
    return []

def process_literal(exp, scope):
    return []

def process_member_reference(exp, scope):
    if scope.is_tracked(exp.member) is None:
        return []
    else:
        return [exp.member]

def process_local_variable_declaration(exp, scope):
    variables = exp.children[3]
    for var in variables:
        process_variable_declarator(var, scope, exp.type.name)
        
def process_class_creator(exp, scope):
    affected = []
    for arg in exp.arguments:
        affected.extend(process_expression(arg, scope))
    return set(affected)
            
# t stands for type
def process_variable_declarator(exp, scope, t = None):
    variable = VariableAlias(exp.name, t)
    scope.add_local_variable(variable)
    affected = process_expression(exp.initializer, scope)
    
    for var in set.intersection(set(affected), scope.get_trackable()):
        variable.add_parent(scope.get_local_variable(var))
        scope.make_trackable(variable.name)
    for var in affected:
        variable.add_parent(scope.get_local_variable(var))
        
def process_statement_expression(exp, scope):
    return process_expression(exp.expression, scope)

def process_method_invocation(exp, scope):
    affected = []
    for arg in exp.arguments:
        affected.extend(process_expression(arg, scope))
    t = set.intersection(set(affected), scope.get_trackable())
    for var in affected:
        lv = scope.get_local_variable(var)
        lv.change_gc(Variable_GC.pass_forward)
    
def process_return_statemnet(exp, scope):
    affected = process_expression(exp.expression, scope)
    t = set.intersection(set(affected), scope.get_trackable())
    
    for var in affected:
        lv = scope.get_local_variable(var)
        lv.change_gc(Variable_GC.not_garbage)

In [477]:
program = read_program("program.txt")
tree = javalang.parse.parse(program)
class_table = get_class_table(tree)

In [478]:
for var in class_table['HelloWorld'].methods['main'].scope.local_variables.items():
    print(var[0], var[1].gc)

df Variable_GC.garbage
args Variable_GC.garbage
f Variable_GC.not_garbage
b Variable_GC.not_garbage
c Variable_GC.not_garbage
a Variable_GC.not_garbage


In [479]:
for var in class_table['HelloWorld'].methods['main'].scope.tracked_variables.items():
    print(var[0], var[1].gc)

b Variable_GC.not_garbage
args Variable_GC.garbage
c Variable_GC.not_garbage
a Variable_GC.not_garbage


In [468]:
for i in class_table['HelloWorld'].methods['main'].scope.local_variables['b'].parents:
    print(i.name)

a


In [434]:
path = tree.types[0].methods[2].body[1]
print(path)
list(filter(lambda x: x[0] != '_', dir(path)))

ReturnStatement


['attrs', 'children', 'expression', 'filter', 'label', 'position']